In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
import tensorflow.keras
from tensorflow.keras import backend as K
from imblearn.over_sampling import SMOTE

In [5]:
dataset = pd.read_excel('anti-bacterial_test.xlsx')
y_test = dataset['label']
y_test = np.array(y_test)

In [6]:
# assign the dataset 
X_train_data_name = 'anti-bacterial_esm2_train.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'anti-bacterial_esm2_test.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)


# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Activation, multiply, add, GlobalAveragePooling1D, Flatten,Reshape
from tensorflow.keras.models import Sequential,Model

In [9]:
def channel_attention(input_feature, reduction_ratio=8): # 定义通道注意力
    channel = input_feature.shape[-1]
    avg_pool = GlobalAveragePooling1D()(input_feature)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    avg_pool = Conv1D(channel//reduction_ratio, (3), padding='same', activation='relu')(avg_pool)
    avg_pool = Conv1D(channel, (1), padding='same')(avg_pool)
#     max_pool = tf.keras.layers.GlobalMaxPooling1D()(input_feature)
    max_pool = Reshape((1, 1, channel))(avg_pool)
    max_pool = Conv1D(channel//reduction_ratio, (3), padding='same', activation='relu')(max_pool)
    max_pool = Conv1D(channel, (1), padding='same')(max_pool)
    scale = add([avg_pool, max_pool])
    scale = Activation('sigmoid')(scale)
    return multiply([input_feature, scale])

def spatial_attention(input_feature): # 定义空间注意力 kernel_size = 7
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(input_feature)
    max_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.max(x, axis=3, keepdims=True))(input_feature)
    concat = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])
    concat = Conv1D(1, (3), padding='same', activation='sigmoid')(concat)
    return multiply([input_feature, concat])

def cbam_block(cbam_feature): # 定义CBAM注意力块
    cbam_feature = channel_attention(cbam_feature)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def build_model(X_train, y_train, X_test, y_test): # 手写数字分类任务简单的CNN模型
    input = tf.keras.layers.Input(shape=(320,1))
    conv1 = Conv1D(32, (3), strides=(1), padding='same', activation='relu')(input) #(B,320,32)
    conv1 = BatchNormalization()(conv1)
#     conv1 = MaxPooling1D((2), name='MaxPool1',padding="same")(conv1)
    conv2 = Conv1D(64, (3), strides=(1), padding='same', activation='relu')(conv1)  #(B,320,64)
    conv2 = BatchNormalization()(conv2)
#     conv2 = MaxPooling1D((2), name='MaxPool2',padding="same")(conv2)
    conv3 = Conv1D(128, (3), strides=(1), padding='same', activation='relu')(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = cbam_block(conv3)  #(B,1,320,128)# 加入CBAM注意力机制
    conv3 = tf.squeeze(conv3,1) #(B,320,128)
    flatten = Flatten()(conv3) #(B,128)
    x = tf.keras.layers.Dense(units=2, activation='softmax')(flatten)
    model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
#     momentum = 0.9
#     sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
    adam = Adam(learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)
  # compile the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
  # learning deccay setting
    import math
    def step_decay(epoch): # gradually decrease the learning rate
        initial_lrate=0.01
        drop=0.6
        epochs_drop = 3.0
        lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
              math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
        return lrate
    lrate = LearningRateScheduler(step_decay)

  # early stop setting
    early_stop = EarlyStopping(monitor='val_accuracy', patience = 50,restore_best_weights = True)

  # summary the callbacks_list
    callbacks_list = [ lrate , early_stop]

    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=100,callbacks=callbacks_list,batch_size = 16, verbose=1)
    return model, model_history

In [10]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 5 
kf = KFold(n_splits=k, shuffle = True, random_state=3407)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = build_model(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidan

Epoch 1/100
140/140 [==============================] - 8s 44ms/step - loss: 0.8495 - accuracy: 0.8993 - val_loss: 0.8303 - val_accuracy: 0.5438 - lr: 0.0100
Epoch 2/100
140/140 [==============================] - 6s 41ms/step - loss: 0.1183 - accuracy: 0.9539 - val_loss: 0.7351 - val_accuracy: 0.5492 - lr: 0.0100
Epoch 3/100
140/140 [==============================] - 6s 46ms/step - loss: 0.0814 - accuracy: 0.9691 - val_loss: 0.3411 - val_accuracy: 0.8283 - lr: 0.0060
Epoch 4/100
140/140 [==============================] - 6s 45ms/step - loss: 0.0670 - accuracy: 0.9740 - val_loss: 0.1691 - val_accuracy: 0.9249 - lr: 0.0060
Epoch 5/100
140/140 [==============================] - 6s 45ms/step - loss: 0.0649 - accuracy: 0.9740 - val_loss: 0.0954 - val_accuracy: 0.9678 - lr: 0.0060
Epoch 6/100
140/140 [==============================] - 6s 45ms/step - loss: 0.0419 - accuracy: 0.9839 - val_loss: 0.0994 - val_accuracy: 0.9571 - lr: 0.0036
Epoch 7/100
140/140 [==============================] - 6s 

Epoch 52/100
140/140 [==============================] - 7s 51ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.1084 - val_accuracy: 0.9660 - lr: 1.6927e-06
Epoch 53/100
140/140 [==============================] - 7s 51ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1082 - val_accuracy: 0.9642 - lr: 1.6927e-06
Epoch 54/100
140/140 [==============================] - 7s 51ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1084 - val_accuracy: 0.9660 - lr: 1.0156e-06
Epoch 55/100
140/140 [==============================] - 7s 50ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1085 - val_accuracy: 0.9660 - lr: 1.0156e-06
Epoch 56/100
140/140 [==============================] - 8s 55ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1082 - val_accuracy: 0.9642 - lr: 1.0156e-06
Epoch 57/100
140/140 [==============================] - 7s 51ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1085 - val_accuracy: 0.9660 - lr: 6.0936e-07
Epoch 58/100
140/140 [======

D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


Epoch 1/100
140/140 [==============================] - 10s 53ms/step - loss: 0.2334 - accuracy: 0.9029 - val_loss: 0.6837 - val_accuracy: 0.5420 - lr: 0.0100
Epoch 2/100
140/140 [==============================] - 7s 51ms/step - loss: 0.1270 - accuracy: 0.9485 - val_loss: 0.1705 - val_accuracy: 0.9338 - lr: 0.0100
Epoch 3/100
140/140 [==============================] - 7s 48ms/step - loss: 0.0949 - accuracy: 0.9673 - val_loss: 0.2957 - val_accuracy: 0.9177 - lr: 0.0060
Epoch 4/100
140/140 [==============================] - 7s 48ms/step - loss: 0.0561 - accuracy: 0.9812 - val_loss: 0.1424 - val_accuracy: 0.9481 - lr: 0.0060
Epoch 5/100
140/140 [==============================] - 6s 46ms/step - loss: 0.0452 - accuracy: 0.9848 - val_loss: 0.1294 - val_accuracy: 0.9553 - lr: 0.0060
Epoch 6/100
140/140 [==============================] - 7s 50ms/step - loss: 0.0234 - accuracy: 0.9911 - val_loss: 0.1175 - val_accuracy: 0.9624 - lr: 0.0036
Epoch 7/100
140/140 [==============================] - 7s

...

D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


Epoch 1/100
140/140 [==============================] - 9s 50ms/step - loss: 0.4252 - accuracy: 0.9141 - val_loss: 1.0513 - val_accuracy: 0.5349 - lr: 0.0100
Epoch 2/100
140/140 [==============================] - 7s 51ms/step - loss: 0.1024 - accuracy: 0.9633 - val_loss: 0.3776 - val_accuracy: 0.8909 - lr: 0.0100
Epoch 3/100
140/140 [==============================] - 7s 52ms/step - loss: 0.0546 - accuracy: 0.9799 - val_loss: 0.3144 - val_accuracy: 0.8515 - lr: 0.0060
Epoch 4/100
140/140 [==============================] - 6s 44ms/step - loss: 0.0445 - accuracy: 0.9834 - val_loss: 0.1434 - val_accuracy: 0.9499 - lr: 0.0060
Epoch 5/100
140/140 [==============================] - 6s 43ms/step - loss: 0.0339 - accuracy: 0.9884 - val_loss: 0.1638 - val_accuracy: 0.9410 - lr: 0.0060
Epoch 6/100
140/140 [==============================] - 6s 45ms/step - loss: 0.0207 - accuracy: 0.9924 - val_loss: 0.1116 - val_accuracy: 0.9660 - lr: 0.0036
Epoch 7/100
140/140 [==============================] - 7s 

D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


Epoch 1/100
140/140 [==============================] - 9s 53ms/step - loss: 0.9903 - accuracy: 0.8900 - val_loss: 0.7046 - val_accuracy: 0.5036 - lr: 0.0100
Epoch 2/100
140/140 [==============================] - 7s 50ms/step - loss: 0.1431 - accuracy: 0.9441 - val_loss: 0.7599 - val_accuracy: 0.5036 - lr: 0.0100
Epoch 3/100
140/140 [==============================] - 7s 48ms/step - loss: 0.1019 - accuracy: 0.9597 - val_loss: 0.4513 - val_accuracy: 0.7168 - lr: 0.0060
Epoch 4/100
140/140 [==============================] - 7s 53ms/step - loss: 0.0949 - accuracy: 0.9647 - val_loss: 0.1354 - val_accuracy: 0.9480 - lr: 0.0060
Epoch 5/100
140/140 [==============================] - 7s 48ms/step - loss: 0.0760 - accuracy: 0.9718 - val_loss: 0.0758 - val_accuracy: 0.9659 - lr: 0.0060
Epoch 6/100
140/140 [==============================] - 7s 47ms/step - loss: 0.0600 - accuracy: 0.9790 - val_loss: 0.0491 - val_accuracy: 0.9749 - lr: 0.0036
Epoch 7/100
140/140 [==============================] - 7s 

KeyboardInterrupt: 

In [ ]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = build_model(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)